*Antonio Coín Castro     
INE*

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.precision", 3)
np.set_printoptions(precision=3)

# Práctica 4: relevance feedback

Consideramos de nuevo la consulta $q_1$ de la práctica 2, y la matriz término-documento que tenemos para ella de la práctica 3.

*$q_1$ = "cuánto cobra jugador profesional fútbol sala"*.

## Ejercicio 1 

*Obtener un nuevo ránking a partir de un ránking inicial, utilizando una iteración la técnica de relevance feedback*.

Para ello debemos seguir el siguiente procedimiento.

1. Partimos de un ránking inicial, a elegir entre:
    - Ránking de Google inicial.
    - **Ránking dado por un modelo vectorial (tf-idf).**
    - Ránking dado por un modelo probabilístico.
2. Técnica a usar: **expansión de consulta por el método Rocchio y rsv**, eligiendo convenientemente el tamaño de rsv y la variante de Rocchio junto a los parámetros $\alpha,\beta$ y $\gamma$.
3. Elegir una fuente de feedback:
    - Usar clicks como relevancia (consulta con qid 6 del documento de clicks). Debemos quedarnos solo con los documentos que coincidan con los nuestros (aunque otra opción es expandir la matriz a mano añadiento nuevos documentos.). También debemos decidir si a los documentos no clickados se les asigna "no relevantes". 
    - Pseudo-relevance feedback: considerar que el top n del ránking inicial es el conjunto de relevante, y que el bottom n es el de no relevantes.
    - **Utilizar directamente los juicios de relevancia (qrels) de la práctica 2.**
    
Una vez que tenemos la nueva consulta $\boldsymbol{q}'$, volvemos a calcular el ránking a partir de ella usando la misma técnica que en el ránking inicial.

---

En primer lugar cargamos la matriz de frecuencias término-documento, añadiendo también los juicios de relevancia de la práctica 2.

In [2]:
# Load document data
doc_term = pd.read_csv(
    "../P3/data/document-term.csv",
    header=0
)

# Get qrels and merge with document data
qrels = pd.read_csv(
    "../P2/data/qrels.csv",
    usecols=[0, 2, 3],
    header=0,
    names=["id", "url", "relevance"]
)
qrels = qrels[qrels['id'] == 1][['url', 'relevance']]

# Create reference dataframe
df_freq = doc_term.merge(qrels, on=['url'], how='outer')
df_freq.drop('url', axis=1, inplace=True)

# Save useful information
terms = list(df_freq.columns[1:-1])
n_terms = len(terms)
D = df_freq.shape[0]
relevant_docs = df_freq['relevance'] > 0

# Load query data
q_text = "cuánto cobra jugador profesional fútbol sala"
q_words = q_text.split(' ')
q = np.array([1 if w in q_words else 0 for w in terms])

df_freq.style.hide_index()

id,aficionado,cláusula,cobra,convenio,cuánto,deporte,división,élite,español,fútbol,honor,jugador,mensual,mileurista,mínimo,prima,profesional,renombre,sala,salario,relevance
1,0,4,1,0,0,2,1,1,3,16,0,9,4,1,3,0,5,0,11,4,2
2,1,0,1,3,3,1,1,0,1,10,1,4,3,0,5,1,4,1,9,7,2
3,0,0,0,0,6,5,0,0,7,37,0,23,1,0,1,0,3,0,11,5,1
4,1,0,1,5,6,6,0,0,1,19,2,19,1,0,2,0,2,1,25,7,2
5,0,0,4,0,16,2,5,0,1,8,0,9,2,0,4,2,2,0,0,5,0
6,0,0,9,0,24,0,9,2,29,35,0,26,1,0,5,0,5,0,0,5,0
7,0,0,1,0,1,0,5,0,0,23,0,7,1,0,0,0,2,0,1,0,0
8,0,0,7,0,6,3,4,0,9,26,0,9,2,0,3,0,2,0,1,4,0
9,0,0,6,3,1,0,7,0,3,15,0,11,6,0,6,4,11,0,0,4,0
10,0,0,3,3,2,0,4,0,3,7,0,3,1,0,6,2,2,0,0,4,0


Convertimos esta matriz en una matriz tf-idf (modelo vectorial) siguiendo las indicaciones de la práctica 3, donde aplicamos directamente un suavizado en idf para evitar casos anómalos. Concretamente, las fórmulas que usamos son:

$$
\operatorname{tf}(w, d) = \begin{cases} 1 + \log_2 \text{freq}(w, d), & \text{si freq}(w, d) > 0,\\
0, & \text{en otro caso}.\end{cases},
$$

$$
idf(w)=\log \frac{|\mathcal D| + 1}{|\mathcal D_w|+0.5},
$$

donde $\mathcal D$ es el conjunto de todos los documentos y $\mathcal D_w$ el conjunto de los documentos que contienen el término $w$.

In [3]:
def get_vsm_ranking(df, q):
    # Compute tf and idf for the documents
    tf = np.array([
        [1 + np.log2(df[w][d]) if df[w][d] > 0 else 0
         for d in range(D)] for w in terms])
    idf = np.array([np.log10((D + 1)/(sum(df[w] > 0) + 0.5))
                    for w in terms])

    # Compute coordinates of documents
    docs = (tf*idf[:, None]).T

    # Compute scores
    scores = np.array([(d@q)/np.linalg.norm(d) for d in docs])

    return docs, scores

In [4]:
docs, scores = get_vsm_ranking(df_freq, q)

df = df_freq.copy()
df.iloc[:, 1:-1] = docs

df.style.hide_index()

id,aficionado,cláusula,cobra,convenio,cuánto,deporte,división,élite,español,fútbol,honor,jugador,mensual,mileurista,mínimo,prima,profesional,renombre,sala,salario,relevance
1,0.000,2.334,0.079,0.000,0.000,0.264,0.161,0.669,0.271,0.052,0.000,0.044,0.483,1.146,0.582,0.000,0.183,0.000,0.588,0.238,2
2,0.582,0.000,0.079,1.015,0.083,0.132,0.161,0.000,0.105,0.045,0.924,0.031,0.416,0.000,0.748,0.582,0.165,0.778,0.550,0.301,2
3,0.000,0.000,0.000,0.000,0.115,0.438,0.000,0.000,0.399,0.065,0.000,0.058,0.161,0.000,0.225,0.000,0.142,0.000,0.588,0.263,1
4,0.582,0.000,0.079,1.305,0.115,0.473,0.000,0.000,0.105,0.055,1.849,0.055,0.161,0.000,0.451,0.000,0.110,0.778,0.744,0.301,2
5,0.000,0.000,0.238,0.000,0.161,0.264,0.534,0.000,0.105,0.042,0.000,0.044,0.322,0.000,0.676,1.164,0.110,0.000,0.000,0.263,0
6,0.000,0.000,0.330,0.000,0.180,0.000,0.671,1.338,0.614,0.064,0.000,0.060,0.161,0.000,0.748,0.000,0.183,0.000,0.000,0.263,0
7,0.000,0.000,0.079,0.000,0.032,0.000,0.534,0.000,0.000,0.058,0.000,0.040,0.161,0.000,0.000,0.000,0.110,0.000,0.132,0.000,0
8,0.000,0.000,0.301,0.000,0.115,0.341,0.483,0.000,0.437,0.060,0.000,0.044,0.322,0.000,0.582,0.000,0.110,0.000,0.132,0.238,0
9,0.000,0.000,0.284,1.015,0.032,0.000,0.612,0.000,0.271,0.051,0.000,0.047,0.577,0.000,0.808,1.746,0.245,0.000,0.000,0.238,0
10,0.000,0.000,0.205,1.015,0.064,0.000,0.483,0.000,0.271,0.040,0.000,0.027,0.161,0.000,0.808,1.164,0.110,0.000,0.000,0.238,0


Ahora, construimos el ránking de partida dado por el modelo vectorial.

In [5]:
vsm_ranking = scores.argsort()[::-1]
scores_orig = scores[vsm_ranking]

df_vsm = pd.DataFrame({
    'id': vsm_ranking + 1,
    'score': scores_orig}
)

print("Ránking inicial")
df_vsm.style.hide_index()

Ránking inicial


id,score
16,1.757
15,1.278
20,1.123
3,1.030
14,0.836
11,0.823
7,0.759
8,0.702
19,0.560
13,0.514


Para aplicar *relevance feedback* elegimos el método de Rocchio standard, tomando $\alpha=\beta=\gamma=1$. Usaremos los juicios de relevancia de la práctica 2, considerando que los documentos con relevancia estrictamente positiva son los relevantes (conjunto $D_r$), y los que tienen relevancia 0 son los no relevantes (conjunto $D_n$). La expresión de la consulta modificada es entonces:

$$
\boldsymbol{q_m} = \alpha \boldsymbol{q} + \frac{\beta}{|D_r|}\sum_{\boldsymbol d\in D_r}\boldsymbol{d} - \frac{\gamma}{|D_n|}\sum_{\boldsymbol d\in D_n}\boldsymbol{d}.
$$

In [6]:
def get_qm(df, relevant_docs, q, alpha=1, beta=1, gamma=1):
    mean_rel = np.mean(df[relevant_docs]).values[1:-1]
    mean_no_rel = np.mean(df[~relevant_docs]).values[1:-1]
    qm = alpha*q + beta*mean_rel - gamma*mean_no_rel
    return qm

In [7]:
qm = get_qm(df, relevant_docs, q)
print(f"Nueva consulta qm:\n{qm}")

Nueva consulta qm:
[ 0.267  0.162  0.878  0.02   0.993  0.159 -0.385 -0.056 -0.147  0.998
  0.347  1.006  0.031  0.143 -0.16  -0.315  1.008  0.292  1.357 -0.002]


Una vez que tenemos nuestra consulta modificada, procedemos a seleccionar solo algunos términos mediante el método de *Robertson Selection Value (rsv)*, que recordemos se expresa como

$$
rsv(w) = |r_w|\cdot BIR(w),
$$

donde

$$
BIR(w)=\log \frac{(|r_w|+0.5)(|\mathcal D|- |\mathcal D_w| - |r| + |r_w| + 0.5)}{(|\mathcal D_w| - |r_w| + 0.5)(|r|-|r_w|+0.5)}.
$$

Las variables que intervienen son:

- $r$ es el conjunto de documentos relevantes,
- $r_w$ es el conjunto de documentos relevantes que contienen la palabra $w$,
- $\mathcal D$ es el conjunto de todos los documentos considerados, y
- $\mathcal D_w$ es el conjunto de documentos que contienen a $w$.

In [8]:
def rsv_terms(df, relevant_docs):
    R = sum(relevant_docs)
    rsv = np.zeros(len(terms))

    for i, w in enumerate(terms):
        Dw = sum(df[w] > 0)
        rw = sum(df[w][relevant_docs] > 0)
        rsv[i] = rw*(np.log10((rw + 0.5)/(R - rw + 0.5))
                     + np.log10((D - Dw + rw - R + 0.5)/(Dw - rw + 0.5)))
    rsv_rank = rsv.argsort()[::-1]

    return rsv_rank, rsv[rsv_rank]

Una vez que tenemos el valor de $rsv$ para cada término de nuestra colección, tomamos como *cutoff* el top $n$, y solo expandimos las correspondientes componentes de la consulta $\boldsymbol{q}_m$, ignorando además los términos negativos (los ponemos a 0). En nuestro caso, elegimos $n=10$ para considerar solo con la mitad de las componentes.

In [9]:
rsv_rank, rsv = rsv_terms(df_freq, relevant_docs)
top_rsv = 10
qm_red = np.array(
    [qm[i] if i in rsv_rank[:top_rsv] else 0
     for i in range(len(qm))]
)
# Ignore negative terms
qm_red[qm_red < 0] = 0

print(f"Nueva consulta reducida q':\n{qm_red}")

Nueva consulta reducida q':
[0.267 0.    0.    0.    0.    0.159 0.    0.    0.    0.    0.347 0.
 0.031 0.143 0.    0.    1.008 0.292 1.357 0.   ]


Finalmente, volvemos a calcular el ránking de los documentos que tenemos en nuestra colección utilizando la consulta reducida, empleando de nuevo el modelo vectorial basado en tf-idf y similitud por ángulo.

In [10]:
_, scores = get_vsm_ranking(df_freq, qm_red)
vsm_ranking_new = scores.argsort()[::-1]
scores_new = scores[vsm_ranking_new]

df_vsm_new = pd.DataFrame({
    'id': vsm_ranking_new + 1,
    'score': scores_new}
)

print("Nuevo ránking")
df_vsm_new.style.hide_index()

Nuevo ránking


id,score
16,1.419
3,1.080
20,1.006
19,0.980
15,0.878
4,0.828
2,0.787
11,0.583
7,0.496
17,0.479


Podemos unir los dos ránkings para visualizarlos a la vez.

In [11]:
df_compare = pd.DataFrame({
    'rank_orig': vsm_ranking + 1,
    'score_orig': scores_orig,
    'rank_new': vsm_ranking_new + 1,
    'score_new': scores_new}
)

df_compare.style.hide_index()

rank_orig,score_orig,rank_new,score_new
16,1.757,16,1.419
15,1.278,3,1.080
20,1.123,20,1.006
3,1.030,19,0.980
14,0.836,15,0.878
11,0.823,4,0.828
7,0.759,2,0.787
8,0.702,11,0.583
19,0.560,7,0.496
13,0.514,17,0.479


Vemos que el primer documento del ránking sigue siendo el mismo, pero observamos algunas variaciones tanto en la parte de arriba como en la parte de abajo del ránking. Un caso curioso es el del documento 14, que antes se encontraba en la mitad superior del ránking y ahora ha pasado a ser el último, con una puntuación de 0.

## Ejercicio 2

*Comparar la precisión y el nDCG del ránking inicial y del nuevo ránking obtenido en el ejercicio anterior. Analizar y comentar los resultados*.

En primer lugar unimos los ránkings obtenidos con sus juicios de relevancia.

In [12]:
df_orig = df_vsm.merge(
    df_freq, on=['id'], how='outer')[['id', 'relevance']]
df_new = df_vsm_new.merge(
    df_freq, on=['id'], how='outer')[['id', 'relevance']]

Ahora calculamos las métricas. Recordamos que la precisión se define como

$$
P = \frac{|Ret \cap Rel|}{|Ret|},
$$

donde $Ret$ es el conjunto de documentos devueltos y $Rel$ el conjunto de documentos relevantes. Para poder hacer comparaciones justas, consideramos la precisión por ejemplo $@10$. De esta forma no obtendremos siempre todos los documentos relevantes a la hora de calcular esta métrica, y podremos observar las diferencias entre distintos enfoques.

Por otro lado, la métrica nDCG se define como 

$$
nDCG=\frac{DCG}{IDCG}.
$$

Si $g\left(d_k\right)$ representa el grado de relevancia (en nuestro caso en $\{0, 1, 2\}$) del documento $k$-ésimo, entonces

$$
DCG = \sum_{k=1}^{|Ret|} \frac{g\left(d_k\right)}{\log_2(k+1)},
$$

donde notamos que los documentos no relevantes no contribuyen al sumatorio. Por otro lado, la constante de normalización es el DCG ideal, que viene dado por

$$
IDCG = \max_{R\in Rel} DCG(R),
$$

que no es más que el mayor DCG que podríamos conseguir teniendo en cuenta el corpus completo de documentos relevantes para nuestra consulta, ordenado convenientemente de mayor a menor relevancia. Gracias a esta normalización, el valor de $nDCG$ se encuentra en $[0,1]$ y es comparable para diferentes consultas.

In [13]:
def precision(df, cutoff=D):
    return sum(df['relevance'][:cutoff] > 0)/cutoff


def DCG(rel_degrees):
    return np.sum(
        [rel_degrees[d]/np.log2(d + 2)
         for d in range(D)])


def nDCG(df):
    df_sorted = df.sort_values(by=["relevance"], ascending=False)
    IDCG = DCG(df_sorted['relevance'].values)
    return DCG(df['relevance'].values)/IDCG

In [14]:
cutoff = 10

# Original
prec_orig = precision(df_orig, cutoff)
ndcg_orig = nDCG(df_orig)

# Nuevo
prec_new = precision(df_new, cutoff)
ndcg_new = nDCG(df_new)

In [15]:
print(f"Precisión (@10) ránking original: {prec_orig:.4f}")
print(f"Precisión (@10) ránking nuevo: {prec_new:.4f}")
print("---")
print(f"nDCG ránking original: {ndcg_orig:.4f}")
print(f"nDCG ránking nuevo: {ndcg_new:.4f}")

Precisión (@10) ránking original: 0.4000
Precisión (@10) ránking nuevo: 0.7000
---
nDCG ránking original: 0.8394
nDCG ránking nuevo: 0.8757


Vemos que con el nuevo ránking hemos conseguido mejorar ambas métricas. Esto tiene sentido, ya que tras la iteración de *relevance feedback* estamos incorporando al procedimiento de ránking información de relevancia, haciendo que en el nuevo ránking se tienda a mostrar primero los resultados más relevantes para el usuario.

Como último comentario, notamos que la opción de utilizar los juicios de relevancia no corresponde a un escenario real (donde el feedback lo daría un usuario final en lugar de un asesor) ni a una evaluación “justa” en laboratorio (pues le enseñamos al sistema las “soluciones” a la pregunta sobre la que va a ser evaluado), pero sí que sirve como prueba de contraste para establecer un *baseline* a la hora de comparar otras opciones.

## Ejercicio opcional 

*Probar alguna o algunas de las siguientes propuestas:*

- *Cambiar los parámetros $\alpha, \beta$ y $\gamma$ y/o la variante de Rocchio utilizada.*
- *Variar el cutoff en BIR o el tamaño de rsv.*
- *Hacer más iteraciones de relevance feedback.*
- *Probar con  técnicas diferentes y/o distintos ránkings iniciales y fuentes de feedback.*

### Más iteraciones de relevance feedback

Procedemos a hacer una segunda iteración mediante el mismo procedimiento de antes, y comparamos las métricas del nuevo ránking para ver si seguimos mejorando. Notamos que no varía ni la matriz tf-idf ni el conjunto de documentos relevantes; tan solo evoluciona la consulta.

In [16]:
def iterate_rel_feedback(
    df,
    relevant_docs,
    n_iter,
    q_init,
    n=10,
    alpha=1,
    beta=1,
    gamma=1,
    verbose=False
):
    q = q_init
    
    print(f"[Iteración 0] Precisión (@10) ránking: {prec_orig:.4f}")
    print(f"[Iteración 0] nDCG ránking: {ndcg_orig:.4f}")
    print("-----")

    for i in range(n_iter):
        qm = get_qm(df, relevant_docs, q, alpha, beta, gamma)
        rsv_rank, _ = rsv_terms(df_freq, relevant_docs)
        qm_red = np.array(
            [qm[j] if j in rsv_rank[:n] else 0
             for j in range(len(qm))]
        )
        # Ignore negative weights
        qm_red[qm_red < 0] = 0

        if verbose:
            print(
                f"[Iteración {i + 1}] Nueva consulta reducida q':\n{qm2_red}")

        _, scores = get_vsm_ranking(df_freq, qm_red)
        ranking = scores.argsort()[::-1]
        scores = scores[ranking]

        df_vsm = pd.DataFrame({
            'id': ranking + 1,
            'score': scores}
        )
        df_new = df_vsm.merge(
            df_freq, on=['id'], how='outer')[['id', 'relevance']]
        prec_new = precision(df_new, cutoff)
        ndcg_new = nDCG(df_new)

        print(f"[Iteración {i + 1}] Precisión (@10) ránking: {prec_new:.4f}")
        print(f"[Iteración {i + 1}] nDCG ránking: {ndcg_new:.4f}")
        print("---")

        q = qm

    return df_new

In [17]:
df_final = iterate_rel_feedback(df, relevant_docs, 3, q)

[Iteración 0] Precisión (@10) ránking: 0.4000
[Iteración 0] nDCG ránking: 0.8394
-----
[Iteración 1] Precisión (@10) ránking: 0.7000
[Iteración 1] nDCG ránking: 0.8757
---
[Iteración 2] Precisión (@10) ránking: 0.7000
[Iteración 2] nDCG ránking: 0.9023
---
[Iteración 3] Precisión (@10) ránking: 0.7000
[Iteración 3] nDCG ránking: 0.9237
---


Vemos que no conseguimos aumentar la precisión, pero la métrica de nDCG sí que aumenta conforme hacemos más iteraciones. Esto puede ser porque estamos refinando cada vez más la consulta, haciendo que los resultados relevantes estén cada vez más arriba en el ránking (aunque debemos tener cuidado de no pasarnos y que empiecen a empeorar los resultados). Como prueba de ello, mostramos cómo queda el ránking tras las tres iteraciones en términos de relevancia de documentos.

In [18]:
df_final.style.hide_index()

id,relevance
16,2
19,1
4,2
3,1
2,2
20,0
15,2
17,1
11,0
12,0


### Variar parámetros en Rocchio y número de términos en rsv

Probamos a poner $\gamma<\beta$, haciendo la hipótesis de que los documentos relevantes tienen más importancia que los no relevantes. Mantenemos $\alpha=1$, y elegimos por ejemplo $\beta=0.8$ y $\gamma=0.5$.

In [19]:
df_final = iterate_rel_feedback(
    df,
    relevant_docs,
    n_iter=3,
    q_init=q,
    alpha=1,
    beta=0.8,
    gamma=0.5)

[Iteración 0] Precisión (@10) ránking: 0.4000
[Iteración 0] nDCG ránking: 0.8394
-----
[Iteración 1] Precisión (@10) ránking: 0.7000
[Iteración 1] nDCG ránking: 0.8757
---
[Iteración 2] Precisión (@10) ránking: 0.7000
[Iteración 2] nDCG ránking: 0.8896
---
[Iteración 3] Precisión (@10) ránking: 0.7000
[Iteración 3] nDCG ránking: 0.9048
---


En este caso no conseguimos mejorar ninguna de las métricas (de hecho nDCG empeora en algunos casos). Esto puede ocurrir porque tenemos muchos documentos no relevantes, y quizás la hipótesis de que tienen menos importancia no es adecuada. 

---

Probamos ahora también a variar el parámetro de cutoff en rsv, recuperando los valores de $\alpha,\beta$ y $\gamma$ por defecto (que daban mejores resultados). Por ejemplo, hacemos pruebas eligiendo $n=5$ y $n=15$.

In [20]:
n = 5
df_final = iterate_rel_feedback(
    df,
    relevant_docs,
    n_iter=3,
    q_init=q,
    n=n)

[Iteración 0] Precisión (@10) ránking: 0.4000
[Iteración 0] nDCG ránking: 0.8394
-----
[Iteración 1] Precisión (@10) ránking: 0.6000
[Iteración 1] nDCG ránking: 0.8680
---
[Iteración 2] Precisión (@10) ránking: 0.7000
[Iteración 2] nDCG ránking: 0.8763
---
[Iteración 3] Precisión (@10) ránking: 0.7000
[Iteración 3] nDCG ránking: 0.8874
---


In [21]:
n = 15
df_final = iterate_rel_feedback(
    df,
    relevant_docs,
    n_iter=3,
    q_init=q,
    n=n)

[Iteración 0] Precisión (@10) ránking: 0.4000
[Iteración 0] nDCG ránking: 0.8394
-----
[Iteración 1] Precisión (@10) ránking: 0.7000
[Iteración 1] nDCG ránking: 0.8784
---
[Iteración 2] Precisión (@10) ránking: 0.8000
[Iteración 2] nDCG ránking: 0.9056
---
[Iteración 3] Precisión (@10) ránking: 0.8000
[Iteración 3] nDCG ránking: 0.9245
---


Vemos que elegir un número de términos de expansión más pequeño en rsv hace que obtengamos peores resultados, mientras que si aumentamos $n$ hasta 15 obtenemos unos valores ligeramente mejores de las métricas de los que teníamos antes. Sin embargo, al estar considerando más componentes no nulas en la consulta (y haciendo que sea menos *sparse*) estamos potencialmente aumentando el tiempo de cómputo de sucesivas iteraciones, suponiendo que tuviéramos un conjunto de documentos real de gran tamaño.

### Usar blind relevance feedback

Finalmente, hacemos una última prueba en la que cambiamos los juicios de relevancia. Concretamente, utilizamos la técnica de *blind relevance feedback*, considerando que el *top $k$* del ránking inicial son documentos relevantes, mientras que el resto no lo son. De esta forma, eliminamos la necesidad de intervención del usuario, a riesgo de falsear los resultados si realmente no se cumple nuestra hipótesis.

Vamos a probar con dos valores distintos para $k$. En concreto, probamos $k=5$ y $k=10$.

In [25]:
def get_new_df(k, ranking):
    df_brf = df.copy()
    relevant_docs = ranking[:k]
    df_brf['relevance'] = 0
    df_brf.loc[relevant_docs, 'relevance'] = 1
    return df_brf

In [23]:
df_brf = get_new_df(5, vsm_ranking)
relevant_docs_brf = df_brf['relevance'] > 0
df_final = iterate_rel_feedback(
    df_brf,
    relevant_docs_brf,
    n_iter=3,
    q_init=q)

[Iteración 0] Precisión (@10) ránking: 0.4000
[Iteración 0] nDCG ránking: 0.8394
-----
[Iteración 1] Precisión (@10) ránking: 0.6000
[Iteración 1] nDCG ránking: 0.8490
---
[Iteración 2] Precisión (@10) ránking: 0.7000
[Iteración 2] nDCG ránking: 0.8519
---
[Iteración 3] Precisión (@10) ránking: 0.7000
[Iteración 3] nDCG ránking: 0.8519
---


In [24]:
df_brf = get_new_df(15, vsm_ranking)
relevant_docs_brf = df_brf['relevance'] > 0
df_final = iterate_rel_feedback(
    df_brf,
    relevant_docs_brf,
    n_iter=3,
    q_init=q)

[Iteración 0] Precisión (@10) ránking: 0.4000
[Iteración 0] nDCG ránking: 0.8394
-----
[Iteración 1] Precisión (@10) ránking: 0.5000
[Iteración 1] nDCG ránking: 0.8184
---
[Iteración 2] Precisión (@10) ránking: 0.6000
[Iteración 2] nDCG ránking: 0.8323
---
[Iteración 3] Precisión (@10) ránking: 0.6000
[Iteración 3] nDCG ránking: 0.8587
---


Vemos que obtenemos resultados un poco peores de las métricas, y peores aún si aumentamos $k$ hasta 15 (ya que estamos falseando demasiado la relevancia de los documentos). Sin embargo, los resultados no están tan lejos de los que obteníamos con los juicios de relevancia, y a cambio no es necesaria la intervención del usuario para proporcionar feedback, cosa de la que a veces no disponemos en la práctica. Por tanto, usar *BRF* en este caso parece un enfoque aceptable.